<a href="https://colab.research.google.com/github/victoriaazabel/SEdigitalorganizations/blob/main/VictoriaZabel_Assignment8_ReinforcedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforced Learning
- For the final assignment I chose the topic reinforced learning because this is a new concept for me and we have worked with simple language models before.

- I chose to work with the alien moon landing at first for a better overview


# Goals
- Use Gymnasium, the environment library
- Use Stable-Baselines3, the deep reinforcement learning library
- Understand the code behind each step presented in presentation

In [2]:
# Step 1: install dependencies to create a virtual screen and see what's going on
!apt install swig cmake

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (4.0.1-5build1).
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [3]:
# Step 2: this will install virtual screen libraries and create and run a virtual screen
!sudo apt-get update
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-2build1).
0 upgraded, 0 newly installed, 0 to r

In [ ]:
# force restart run time for the libaries
import os
os.kill(os.getpid(), 9)

In [4]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [14]:
import gymnasium as gym

In [ ]:
!pip install stable-baselines3

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

With Gymnasium:
1. Create our environment using gymnasium.make()
2. Reset the environment to its initial state with observation = env.reset()


At each step:
3. Get an action using our model (in our example we take a random action)
4. Using env.step(action), we perform this action in the environment and get:


- observation: The new state (st+1)
- reward: The reward we get after executing the action
- terminated: Indicates if the episode terminated (agent reach the terminal state)
- truncated: Introduced with this new version, it indicates a timelimit or if an agent go out of bounds of the environment for instance.
- info: A dictionary that provides additional information (depends on the environment)

In [ ]:
# error cuz this was missing but wasn't mentioned, gymnasiym box2d

In [ ]:
!pip install gymnasium[box2d]

In [ ]:
# create our environment
env = gym.make("LunarLander-v2")

# reset
observation, info = env.reset()

# for random actions, step 3
for _ in range(20):
  action = env.action_space.sample()
  print("Action taken:", action)

  # perform action in the environment and get
  # next_state, reward, terminated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)

  # terminated (in our case we land, crashed) or truncated (timeout)
  if terminated or truncated:
      # Reset the environment
      print("Environment is reset")
      observation, info = env.reset()

env.close()

In [21]:
# agent = alien, goal = land correctly on the moon
# learn to adapt speed and position / up down at angles

#  recreate our environment

env = gym.make("LunarLander-v2")
env.reset()

print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [-31.308905  -58.320457    3.2222536  -1.9791884  -1.9628882  -1.0690985
   0.5332321   0.4256067]


Output: Observation Space Shape (8,) = vector of size 8
Each value contains different information about the lander:

- Horizontal pad coordinate (x)
- Vertical pad coordinate (y)
- Horizontal speed (x)
- Vertical speed (y)
- Angle
- Angular speed
- If the left leg contact point has touched the land
- If the right leg contact point has touched the land

--> Observation Space Shape (8,)
Sample observation [-31.308905  -58.320457    3.2222536  -1.9791884  -1.9628882  -1.0690985
   0.5332321   0.4256067]

In [22]:
# discrete action space: limited to four possibilities
# Action 0: Do nothing,
# Action 1: Fire left orientation engine,
# Action 2: Fire the main engine,
# Action 3: Fire right orientation engine

print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 3


Reward function:
- After every step a reward is granted.
- The total reward of an episode is the sum of the rewards for all the steps within that episode.


For each step, the reward:
- Is increased/decreased the closer/further the lander is to the landing pad.
- Is increased/decreased the slower/faster the lander is moving.
- Is decreased the more the lander is tilted (angle not horizontal).
- Is increased by 10 points for each leg that is in contact with the ground.
- Is decreased by 0.03 points each frame a side engine is firing.
- Is decreased by 0.3 points each frame the main engine is firing.

The episode receive an additional reward of -100 or +100 points for crashing or landing safely respectively.


An episode is considered a solution if it scores at least 200 points

In [ ]:
!pip install git+https://github.com/DLR-RM/stable-baselines3

In [26]:
# from now on we'll need these dumb impossible to import packages


from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [27]:
# vectorized environment created to  stack multiple independent environments into a single environment, mix of 16 environments
# why? --> more diverse experiences during training
env = make_vec_env('LunarLander-v2', n_envs=16)

In [28]:
# use deep RL library to create model
# already created environemtn so now we need the model

# We added some parameters to accelerate the training
model = PPO(
    policy = 'MlpPolicy', # which method
    env = env, # use previous environment
    n_steps = 1024, # interactions  agent will have with the environment before collecting experience to update its policy
    batch_size = 64, # samples that will be used in each training iteration
    n_epochs = 4, # run throughs
    gamma = 0.999, # discount rate, higher so smaller discount = our agent cares more about the long-term reward
    gae_lambda = 0.98, # ???? how much the agent values the current estimate of advantages compared to future estimates?????????
    ent_coef = 0.01, # exploration / exploitation trade off = exploiting here
    verbose=1) # 1 enables the algorithm to print progress information and updates while training.

Using cuda device


In [ ]:
# Training model

# 1,000,000 timesteps for improvement of results

model.learn(total_timesteps=1000000)

model_name = "ppo-LunarLander-v2"
model.save(model_name)

In [32]:
# evaluating agent, not model!
# create an evaluation environment
# Stable-Baselines3 method: evaluate_policy according to rewards collected in evalutaion cuz the goal is to have the highest points / rewards

eval_env = Monitor(gym.make("LunarLander-v2"))

# using prev model, new environment
# evaluation episodes = 10, policy will be evaluated for 10 episodes
# determ = True, the policy will select the actions with the highest probability / = False, value based/ Acts on biggest value to attain goal
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

# mean to two dp, sd range
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=269.36 +/- 15.96156252416331
